In [66]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import models
import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np

import os

In [67]:
import matplotlib.pyplot as plt
%matplotlib inline

In [242]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = dsets.ImageFolder('data/train/', train_transform)
test_data = dsets.ImageFolder('data/test_model/', test_transform)

In [243]:
batch_size = 16

train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(test_data, 
                         batch_size=batch_size,
                         shuffle=True)

In [245]:
def imshow(img, title):
    img = torchvision.utils.make_grid(img, normalize=True)
    npimg = img.numpy()
    fig = plt.figure(figsize = (5, 15))
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [160]:
model = models.inception_v3(pretrained=True)

In [161]:
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

In [162]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 10),
    nn.Linear(10, 6)
)

In [ ]:
model = model.cuda()

In [75]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [76]:
num_epochs = 5

In [77]:
for epoch in range(num_epochs):
    
    total_batch = len(train_data)//batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):
#         print(batch_images)
#         print(batch_labels)
        X = batch_images#.cuda()
        Y = batch_labels#.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))
            
torch.save(model.state_dict(), "modelTorch.pt")

Epoch [1/5], lter [5/316] Loss: 1.9105
Epoch [1/5], lter [10/316] Loss: 1.8023
Epoch [1/5], lter [15/316] Loss: 1.8667
Epoch [1/5], lter [20/316] Loss: 1.8310
Epoch [1/5], lter [25/316] Loss: 1.6655
Epoch [1/5], lter [30/316] Loss: 1.6992
Epoch [1/5], lter [35/316] Loss: 1.6409
Epoch [1/5], lter [40/316] Loss: 1.7015
Epoch [1/5], lter [45/316] Loss: 1.7497
Epoch [1/5], lter [50/316] Loss: 1.7047
Epoch [1/5], lter [55/316] Loss: 1.7003
Epoch [1/5], lter [60/316] Loss: 1.7832
Epoch [1/5], lter [65/316] Loss: 1.7006
Epoch [1/5], lter [70/316] Loss: 1.6477
Epoch [1/5], lter [75/316] Loss: 1.6935
Epoch [1/5], lter [80/316] Loss: 1.6345
Epoch [1/5], lter [85/316] Loss: 1.5078
Epoch [1/5], lter [90/316] Loss: 1.5918
Epoch [1/5], lter [95/316] Loss: 1.5338
Epoch [1/5], lter [100/316] Loss: 1.6711
Epoch [1/5], lter [105/316] Loss: 1.5689
Epoch [1/5], lter [110/316] Loss: 1.5605
Epoch [1/5], lter [115/316] Loss: 1.4960
Epoch [1/5], lter [120/316] Loss: 1.7416
Epoch [1/5], lter [125/316] Loss: 1.

Epoch [4/5], lter [70/316] Loss: 0.5053
Epoch [4/5], lter [75/316] Loss: 0.3720
Epoch [4/5], lter [80/316] Loss: 0.5637
Epoch [4/5], lter [85/316] Loss: 0.5622
Epoch [4/5], lter [90/316] Loss: 0.8412
Epoch [4/5], lter [95/316] Loss: 0.6389
Epoch [4/5], lter [100/316] Loss: 0.4827
Epoch [4/5], lter [105/316] Loss: 0.7669
Epoch [4/5], lter [110/316] Loss: 0.2936
Epoch [4/5], lter [115/316] Loss: 0.3328
Epoch [4/5], lter [120/316] Loss: 0.7674
Epoch [4/5], lter [125/316] Loss: 0.7232
Epoch [4/5], lter [130/316] Loss: 0.3944
Epoch [4/5], lter [135/316] Loss: 0.5200
Epoch [4/5], lter [140/316] Loss: 0.5391
Epoch [4/5], lter [145/316] Loss: 0.6498
Epoch [4/5], lter [150/316] Loss: 0.3991
Epoch [4/5], lter [155/316] Loss: 0.4445
Epoch [4/5], lter [160/316] Loss: 0.3599
Epoch [4/5], lter [165/316] Loss: 0.7480
Epoch [4/5], lter [170/316] Loss: 0.6339
Epoch [4/5], lter [175/316] Loss: 0.3822
Epoch [4/5], lter [180/316] Loss: 0.4783
Epoch [4/5], lter [185/316] Loss: 0.4789
Epoch [4/5], lter [190

In [246]:
model.eval()


correct = 0
total = 0

for images, labels in test_loader:
    
#     images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 91.600000 %


# The scripts below: To be continued

In [192]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs#.to(device)
            labels = labels#.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {classes[preds[j]]}')
                imshow(inputs.cpu().data[j], 'test')

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [231]:
from PIL import Image

frame = cv2.imread('./data/test_model/client008/5.jpg')
def face_detector(img, size=0.5):

    # Convert image to grayscale
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if len(faces) == None:
        print("Empty")
        return img, []
    print("NotEmpty", faces)
    for (x,y,w,h) in faces:
        
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (224, 224))
    return img, roi

# im, f = face_detector(frame)
face = Image.fromarray(frame)
image = Image.fromarray(im)
predict(model,face)

TypeError: predict() missing 2 required positional arguments: 'device' and 'encoder'

##############

In [16]:
classes = ["client001", "client002", "client004", "client006", "client007", "client008"]
classes = os.listdir('./data/test_model')
classes

['client001', 'client002', 'client004', 'client006', 'client007', 'client008']

In [233]:
classes = ["client001", "client002", "client004", "client006", "client007", "client008"]
es, labels = next(iter(test_loader))
images = './data/test_model/client001/2.jpg'
outputs = model(images)

_, predicted = torch.max(outputs.data, 1)
    
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(5)))

title = (' '.join('%5s' % classes[labels[j]] for j in range(5)))
imshow(torchvision.utils.make_grid(images, normalize=True), title)

TypeError: string indices must be integers

In [163]:
# torch.save(model, 'modelTorch.pt')
# model = torch.load('modelTorch.pt')
# print(type(model))
# model.eval()
# 
# load your model architecture/module
# model = Inecption3()
# fill your architecture with the trained weights
model.load_state_dict(torch.load("modelTorch.pt"))

<All keys matched successfully>